In [1]:
import llama

def load_model(model_name):

    LOAD_8BIT = False #True
    BASE_MODEL = model_name

    tokenizer = llama.LlamaTokenizer.from_pretrained(BASE_MODEL)
    model = llama.LlamaForCausalLM.from_pretrained(BASE_MODEL, low_cpu_mem_usage=True, load_in_8bit=LOAD_8BIT, torch_dtype=torch.float16, device_map="auto")

    if "openchat" in model.config._name_or_path:

        model.generation_config.pad_token_id = 0

    return tokenizer, model 

In [2]:
import torch 

tokenizer, model = load_model("openchat/openchat_3.5")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type mistral to instantiate a model of type llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/paul/miniconda3/envs/llm_updated/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
for param in HEADS:
    param.requires_grad = True


In [8]:
 model.named_parameters()

<generator object Module.named_parameters at 0x7f141018d660>

In [9]:
for name, param in model.named_parameters():
    if 'model.layers.13.self_attn.head_out' not in name:
    #    param.requires_grad = False
        print(name)
        pass
    else: 
        print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [6]:
for param in model.model.layers[13].self_attn.head_out.parameters():
    param.requires_grad = True
    #print(param)

In [8]:
import pandas as pd 

df = pd.read_json("../requirements_data/dataframe_open_chat_cot_moon_06022024_attentions_gt.json")

df.reset_index(drop=True, inplace=True)
correct = [0 if value == "yes" else 1 for value in df.predict.values]
df.correct = correct

In [10]:
prompt = df.prompt[0]+df.output[0]
prompt_ids = tokenizer(prompt, return_tensors="pt")['input_ids']
prompt_ids = prompt_ids.to('cuda')

output = model(prompt_ids, output_hidden_states = True)#,return_dict_in_generate=True, output_scores=True)

    

OutOfMemoryError: CUDA out of memory. Tried to allocate 34.00 MiB (GPU 0; 23.69 GiB total capacity; 22.23 GiB already allocated; 21.12 MiB free; 22.54 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
i = 13
HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(model.config.num_hidden_layers)]
HEADS = [f"model.layers.{i}.self_attn.head_out"]
HEADS

['model.layers.13.self_attn.head_out']

In [4]:
import inseq

HEADS = [f"model.layers.{i}.self_attn.head_out" for i in range(model.config.num_hidden_layers)]

attrib_model = inseq.load_model(
    model,
    "layer_gradient_x_activation",
    tokenizer=tokenizer,#"lmsys/vicuna-13b-v1.5",#"gpt2-xl",
    #target_layer=model.transformer.h[layer].mlp,
    target_layer = HEADS[0],
   use_attention_weights= True
   
)

In [28]:
inseq.attr.feat.LayerGradientXActivationAttribution

inseq.attr.feat.gradient_attribution.LayerGradientXActivationAttribution

In [6]:
#!pip install inseq --upgrade

In [20]:
#print(df.prompt[0] + df.output[0])

In [7]:
import inseq

attrib_model = inseq.load_model(model, "attention")# "input_x_gradient")#"integrated_gradients")

out = attrib_model.attribute(
    df.prompt[0],
    # generated_texts is used to force the attribution of the specified texts
    generated_texts= df.prompt[0]+df.output[0],
    #step_scores=["probability"],
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Attributing with attention...: 100%|██████████| 1189/1189 [01:26<00:00,  2.44it/s]


In [12]:
data = out.get_scores_dicts()#[0]

In [17]:
data[0].keys()

dict_keys(['source_attributions', 'target_attributions', 'step_scores'])

In [31]:
data[0]['target_attributions'][(1188, '<|end_of_turn|>')][(1174, '▁if')]

0.008493510074913502